In [1]:
import numpy as np
import tensorflow as tf

In [2]:
X = tf.placeholder(tf.float32, shape=(None, None, None), name='X')

In [3]:
X_shape = tf.shape(X)
Xdim0, Xdim1, Xdim2 = X_shape[0], X_shape[1], X_shape[2]

In [4]:
X0 = tf.reshape(tf.transpose(X, perm=(0, 2, 1)), shape=(Xdim0, Xdim1*Xdim2))
X1 = tf.reshape(tf.transpose(X, perm=(1, 2, 0)), shape=(Xdim1, Xdim0*Xdim2))
X2 = tf.reshape(tf.transpose(X, perm=(2, 0, 1)), shape=(Xdim2, Xdim0*Xdim1))

In [5]:
x1 = np.array([[1, 4, 7, 10], [2, 5, 8, 11], [3, 6, 9, 12]], dtype=np.float32)
x2 = np.array([[13, 16, 19, 22], [14, 17, 20, 23], [15, 18, 21, 24]], dtype=np.float32)
x = np.zeros((3, 4, 2))
x[:, :, 0] = x1
x[:, :, 1] = x2

In [6]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
sess.run(X_shape, feed_dict={X: x})
sess.run((Xdim0, Xdim1, Xdim2), feed_dict={X: x})

(3, 4, 2)

In [7]:
sess.run(X0, feed_dict={X: x})

array([[ 1.,  4.,  7., 10., 13., 16., 19., 22.],
       [ 2.,  5.,  8., 11., 14., 17., 20., 23.],
       [ 3.,  6.,  9., 12., 15., 18., 21., 24.]], dtype=float32)

In [8]:
sess.run(X1, feed_dict={X: x})

array([[ 1.,  2.,  3., 13., 14., 15.],
       [ 4.,  5.,  6., 16., 17., 18.],
       [ 7.,  8.,  9., 19., 20., 21.],
       [10., 11., 12., 22., 23., 24.]], dtype=float32)

In [9]:
sess.run(X2, feed_dict={X: x})

array([[ 1.,  4.,  7., 10.,  2.,  5.,  8., 11.,  3.,  6.,  9., 12.],
       [13., 16., 19., 22., 14., 17., 20., 23., 15., 18., 21., 24.]],
      dtype=float32)

In [13]:
# Khatri-Rao product

In [19]:
k = tf.placeholder(tf.int32, name='k')
A0 = tf.Variable(tf.zeros([Xdim0, k]), validate_shape=False, name='A0')
A1 = tf.Variable(tf.zeros([Xdim1, k]), validate_shape=False, name='A1')
A2 = tf.Variable(tf.zeros([Xdim2, k]), validate_shape=False, name='A2')

In [14]:
tmp = tf.constant([1, 2, 3, 4, 5], dtype=tf.float32)
sess.run(tf.foldl(lambda a, x: a+x, tmp, initializer=0.))

15.0

In [52]:
a1 = tf.constant([[2., 3., 3.], [1., 2., -1.]])
a2 = tf.constant([[2., 3., 1.], [1., 2., 3.]])

In [53]:
sess.run(tf.multiply(a1[0, :], a2))

array([[4., 9., 3.],
       [2., 6., 9.]], dtype=float32)

In [67]:
i0 = tf.constant(1)
prod0 = tf.multiply(a1[0, :], a2)
counter, khprod = tf.while_loop(lambda i, m: tf.less(i, tf.shape(a1)[0]),
                                lambda i, m: [i+1, tf.concat([m, tf.multiply(a1[i, :], a2)], 0)],
                                loop_vars=[i0, prod0],
                                shape_invariants=[i0.get_shape(), tf.TensorShape([None, None])])

In [68]:
sess.run(khprod)

array([[ 4.,  9.,  3.],
       [ 2.,  6.,  9.],
       [ 2.,  6., -1.],
       [ 1.,  4., -3.]], dtype=float32)

In [69]:
def khatrirao_product(tensor1, tensor2):
    i0 = tf.constant(1)
    prod0 = tf.multiply(tensor1[0, :], tensor2)
    _, khprod = tf.while_loop(lambda i, m: tf.less(i, tf.shape(tensor1)[0]),
                              lambda i, m: [i+1, tf.concat([m, tf.multiply(tensor1[i, :], tensor2)], 0)],
                              loop_vars=[i0, prod0],
                              shape_invariants=[i0.get_shape(), tf.TensorShape([None, None])])
    return khprod

In [72]:
sess.run(khatrirao_product(a1, a2))

array([[ 4.,  9.,  3.],
       [ 2.,  6.,  9.],
       [ 2.,  6., -1.],
       [ 1.,  4., -3.]], dtype=float32)

# ALS

In [86]:
# we need dimensions in advance
dim0, dim1, dim2 = x.shape
k = 3

In [87]:
A = tf.Variable(initial_value=tf.random_normal([dim0, k]), name='A')
B = tf.Variable(initial_value=tf.random_normal([dim1, k]), name='B')
C = tf.Variable(initial_value=tf.random_normal([dim2, k]), name='C')

In [88]:
costA = tf.reduce_sum(tf.abs(X0 - tf.matmul(A, tf.transpose(khatrirao_product(C, B)))))
costB = tf.reduce_sum(tf.abs(X1 - tf.matmul(B, tf.transpose(khatrirao_product(C, A)))))
costC = tf.reduce_sum(tf.abs(X2 - tf.matmul(C, tf.transpose(khatrirao_product(B, A)))))

In [89]:
alpha = 0.01

trainA = tf.train.GradientDescentOptimizer(alpha).minimize(costA)
trainB = tf.train.GradientDescentOptimizer(alpha).minimize(costB)
trainC = tf.train.GradientDescentOptimizer(alpha).minimize(costC)

In [91]:
init = tf.global_variables_initializer()
sess.run(init)

for _ in range(100):
    sess.run(trainA, feed_dict={X: x})
    sess.run(trainB, feed_dict={X: x})
    sess.run(trainC, feed_dict={X: x})
    
print(sess.run(A, feed_dict={X: x}))
print(sess.run(B, feed_dict={X: x}))
print(sess.run(C, feed_dict={X: x}))

[[ 1.4624751   0.97725594 -1.7744727 ]
 [ 1.7223111   1.2433647  -1.9603282 ]
 [-0.47542602  2.0718288  -1.8354357 ]]
[[ 0.38282987 -0.48583993 -1.9970126 ]
 [-0.19869323 -0.9882004  -1.718226  ]
 [-1.4299908  -1.6319151  -1.8448081 ]
 [-1.2108257  -2.5617292  -1.6278971 ]]
[[-1.111714   -1.8196747   0.53110015]
 [-1.0956985  -1.787748    3.7153187 ]]
